In [1]:
# Imports generales para trabajar con SQL y modelos
import psycopg2
from sqlalchemy import create_engine
from getpass import getpass
import csv
import pandas as pd
import numpy as np

# Imports para ir guardando y leyendo los modelos en la ruta ./Modelos
from os import listdir
import pickle, datetime, os, glob
from pathlib import Path

# Función de auxiliares
from helpers import *

# Modelos
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Parte 1: Registro de los archivos en la base de datos.

### Preparación de la base de datos
A través de Dbeaver ya se generó la database hoffmann_jorge. Debemos conectarnos a ella, de modo que primero almacenamos los datos y generamos la conexión. A posteriori, se preparan las querys de creación.

In [2]:
# Almacenamos los datos de conexión
db_name = "hoffmann_jorge"
host = "localhost"
user = "postgres"
password = getpass("Introducir contraseña")

Introducir contraseña········


In [3]:
# Generamos nuestro objeto conexión
conn = psycopg2.connect(f"dbname={db_name} user={user} password={password}")

# Se crea cursor
cur = conn.cursor()

In [4]:
# Cargamos el dataframe con el objetivo de extraer sus columnas
df_train = pd.read_csv('train_cupid.csv')

In [5]:
# Escribimos la query que nos permitirá crear la tabla en la DB
query_create_table_train = "CREATE TABLE train_cupid("
query_create_table_test = "CREATE TABLE test_cupid("

for col, tipo in zip(df_train.dtypes.index, df_train.dtypes):
    # Reemplazamos espacios y el /.
    col = col.replace(' / ', '').replace(' ', '')
    # Para los tipos float63 e int64 los dejamos en float e int
    tipo = tipo.name.replace('64', '')
    aux = col + ' ' + tipo + ','
    query_create_table_train = query_create_table_train + aux
    query_create_table_test = query_create_table_test + aux
    
query_create_table_train = query_create_table_train[: -1] + "); "
query_create_table_test = query_create_table_test[: -1] + "); "

In [6]:
# Se ejecuta la query usando el cursor
cur.execute(query_create_table_train)
cur.execute(query_create_table_test)

In [7]:
# Commiteamos los cambios
conn.commit()

En este punto, ya generamos las tablas, lo siguiente es ingresar los datos.

In [8]:
with open('train_cupid.csv', 'r') as file:
    # Se lee el contenido del archivo usando csv que permite iterar cada fila
    reader = csv.reader(file)
    # ignoramos la primera fila que corresponde al header
    next(reader)
    # para cada una de las filas remanentes
    for row in reader:
        # ejecutaremos una orden en el cursor que inserte los datos.
        interpoladores = "%s, " * len(row)
        cur.execute(f"INSERT INTO train_cupid VALUES ({interpoladores[:-2]})", row)

In [9]:
with open('test_cupid.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)
    # para cada una de las filas remanentes
    for row in reader:
        interpoladores = "%s, " * len(row)
        cur.execute(f"INSERT INTO test_cupid VALUES ({interpoladores[:-2]})", row)

In [10]:
conn.commit()
conn.close()

# Parte 2: Entrenamiento de modelos

- Ingestar la tabla de training mediante psycopg2 para el posterior entrenamiento del
modelo.
- Entrenar los siguientes modelos (sin necesidad de ajustar por hiper parámetros): GradientBoostingClassifier, AdaBoostClassifer, RandomForestClassifier, SVC, DecisionTreeClassifier,
LogisticRegression, BernoulliNB.
- Existen tres vectores objetivos a evaluar: single, seeing someone y available. Serializar el objeto y preservarlo por cada combinación de modelo entrenado y vector
objetivo.



In [11]:
# Generamos nuestro objeto conexión nuevamente (lo cerramos anteriormente)
conn = psycopg2.connect(f"dbname={db_name} user={user} password={password}")

# Se crea cursor
cur = conn.cursor()

In [12]:
# generamos la consulta en el cursor
cur.execute("SELECT * FROM train_cupid;")

# fetchall permite traer todo lo retornado por la consulta a una variable
# Retorna una lista con tuplas
datos = cur.fetchall()

# Generamos DataFrame
df_train = pd.DataFrame(datos)
cur.execute("Select * FROM train_cupid LIMIT 0;")
columns = [desc[0] for desc in cur.description]
df_train.columns = columns
df_train.head()

,age,height,virgo,taurus,scorpio,pisces,libra,leo,gemini,aries,...,orientation_straight,sex_m,smokes_sometimes,smokes_tryingtoquit,smokes_whendrinking,smokes_yes,body_type_overweight,body_type_regular,education_high_school,education_undergrad_university
0,35,70.0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0
1,38,68.0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0
2,23,71.0,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,1,0,1
3,29,66.0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,1
4,29,67.0,0,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,1


Tenemos nuestros datos en el dataframe df_train obtenidos a través de psycopg2. El siguiente paso es entrenar los modelos

In [13]:
# Generamos una lista de instancias de los modelos
lista_modelos = [
    GradientBoostingClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    LogisticRegression(),
    BernoulliNB(),
    DecisionTreeClassifier(),
    SVC(probability = True)
]

Función train_and_pickle (incluida en helpers.py)
```python
def train_and_pickle(model, X_train, y_train):
    """
        Dado un modelo sklearn, además de X_train e y_train, se entrena el modelo
        y luego se genera un archivo serializado con un nombre identificable.
        
        model: sklearn model class
        X_train: matriz variables independientes
        y_train: vector objetivo
    """
    # Fiteo un modelo con X_train e y_train
    tmp_model_train = model.fit(X_train, y_train)
    # Extraigo el nombre del modelo
    model_name = str(model.__class__).replace("'>", '').split('.')[-1]
    # El nombre del archivo debe ser el VectorObjetivo_Modelo.sav
    nombre = f"{y_train.name}_{model_name}.sav"
    # Se guardará en la ruta ./Modelos
    path = Path("./Modelos")  / nombre
    pickle.dump(tmp_model_train, open(path, 'wb'))
```

In [14]:
# Nos aseguramos de que X tenga toda la data salvo los 3 vectores objetivos
X_train = df_train.copy()
y_train_single = X_train.pop('single')
y_train_seeing_someone = X_train.pop('seeing_someone')
y_train_available = X_train.pop('available')

# Generamos una lista con los vectores objetivo. 
lista_vectores_objetivo = [
    y_train_single,
    y_train_seeing_someone,
    y_train_available
]

In [15]:
# Generamos 21 modelos
for modelo in lista_modelos:
    for vector_objetivo in lista_vectores_objetivo:
        train_and_pickle(modelo, X_train, vector_objetivo)

C:\Users\jahof\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\jahof\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

# Parte 3: Exportación de predicciones
Ingestar la tabla de testing mediante psycopg2 para la posterior predicción del
modelo.  
● En base a los objetos serializados, predecir y evaluar cuatro queries específicas:
- Query 1: 'atheism', 'asian', 'employed', 'pro_dogs', 'chinese'.
- Query 2: 'income_over_75', 'french', 'german','orientation_straight', 'new york'.
- Query 3: 'education_undergrad_university', 'body_type_regular', 'pro_dogs','employed'.
- Query 4: 'taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism'.

In [16]:
# generamos la consulta en el cursor
cur.execute("SELECT * FROM test_cupid;")

# fetchall permite traer todo lo retornado por la consulta a una variable
# Retorna una lista con tuplas
datos = cur.fetchall()

# Generamos DataFrame
df_test = pd.DataFrame(datos)
df_test.columns = columns
df_test.head()

,age,height,virgo,taurus,scorpio,pisces,libra,leo,gemini,aries,...,orientation_straight,sex_m,smokes_sometimes,smokes_tryingtoquit,smokes_whendrinking,smokes_yes,body_type_overweight,body_type_regular,education_high_school,education_undergrad_university
0,22,75.0,0,0,0,0,0,0,1,0,...,1,1,1,0,0,0,0,0,0,1
1,32,65.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,24,67.0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
3,29,62.0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
4,39,65.0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [17]:
conn.close()

In [18]:
# Nos aseguramos de que X tenga toda la data salvo los 3 vectores objetivos
X_test = df_test.copy()
y_test_single = X_test.pop('single')
y_test_seeing_someone = X_test.pop('seeing_someone')
y_test_available = X_test.pop('available')

In [19]:
# Obtengo una lista de todos los modelos guardados
modelos_guardados = [modelo for modelo in listdir(Path("./Modelos"))]
modelos_guardados[: 4]

['single_GradientBoostingClassifier.sav',
 'seeing_someone_GradientBoostingClassifier.sav',
 'available_GradientBoostingClassifier.sav',
 'single_RandomForestClassifier.sav']

Función create_grouped_probabilty:

```python
def create_grouped_probabilty(model, X_test, vector_objetivo, variables):
    """Retorna un DataFrame agrupado por variables que para cada caso tiene la probabilidad
    predicha para la variable de interés del modelo.
    
    model: sklearn model class
    X_test: matriz de variables independientes
    vector_objetivo: string que indica el nombre del vector objetivo
    variables: lista de variables de X_test para el group by
    
    return:
        DataFrame
    """
    tmp = X_test.copy()
    tmp_pr = model.predict_proba(X_test) 

    df_prob = pd.concat([
        tmp.reset_index(drop = True), 
        pd.DataFrame(tmp_pr[:, 1], columns = [vector_objetivo])],
        axis = 1)
    
    tmp_query = df_prob.groupby(variables)[vector_objetivo].mean()
    return tmp_query
```

In [20]:
# Generamos la lista de 'groups by' que necesitaremos más adelante
lista_groups = [
    ['atheism', 'asian', 'employed', 'pro_dogs', 'chinese'],
    ['income_over_75', 'french', 'german','orientation_straight', 'newyork'],
    ['education_undergrad_university', 'body_type_regular', 'pro_dogs','employed'],
    ['taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism']    
]

In [21]:
# Instanciamos engine para ingresar datos a SQL con sqlalchemy
engine = create_engine(f"postgresql://{user}:{password}@{host}/{db_name}")

# Iteramos en la lista de nombres de los 21 modelos
for nombre_modelo in modelos_guardados:
    # Extramos el nombre del vector objetivo separando por '_'.
    # El caso de seeing_someone debe ser tratado especialmente
    nombre_vector_objetivo = nombre_modelo.split('_')[0]
    nombre_vector_objetivo = 'seeing_someone' if nombre_vector_objetivo == 'seeing' else nombre_vector_objetivo
    # A partir del nombre cargamos el modelo
    path_modelo = Path("./Modelos")  / nombre_modelo
    model = pickle.load(open(path_modelo, 'rb')) 
    # Iteramos en cada uno de los groups by que se hacen
    for i, group in enumerate(lista_groups):
        # El nombre de la tabla en SQL será un correlativo de la query más el nombre del modelo sin .sav
        nombre_tabla_sql = f'query_{i + 1}_{nombre_modelo[:-4]}'
        df_grouped = create_grouped_probabilty(model, X_test, nombre_vector_objetivo, group)
        df_grouped.to_sql(
            nombre_tabla_sql,
            con=engine,
            if_exists="replace",
            chunksize=1000,
            method="multi",
            index=True # Por defecto True. Si no se quiere generar columna con el índice, asignar False
        )

In [22]:
engine.dispose()

### sanity check: revisión rendimiento modelos

In [23]:
# Generamos un diccionario con los 3 vectores objetivo
dict_vect_obj_test = {
    'single': y_test_single,
    'seeing_someone': y_test_seeing_someone,
    'available': y_test_available
}

In [24]:
# Revisaremos f1_score para cada uno del os modelos
from sklearn.metrics import f1_score

f1_score_list_class_0 = []
f1_score_list_class_1 = []
f1_score_weighted_list = []
f1_score_macro_list = []


for nombre_modelo in modelos_guardados:
    # Extraemos el nombre del vector objetivo a partir del nombre del modelo
    nombre_vector_objetivo = nombre_modelo.split('_')[0]
    nombre_vector_objetivo = 'seeing_someone' if nombre_vector_objetivo == 'seeing' else nombre_vector_objetivo
    # Obtenemos la serie del vector objetivo
    y_test = dict_vect_obj_test[nombre_vector_objetivo]
    # Con la ruta, cargamos el modelo y predecimos y (el mismo que y_test)
    path_modelo = Path("./Modelos")  / nombre_modelo
    model = pickle.load(open(path_modelo, 'rb'))     
    y_predict = model.predict(X_test)
    # Extraemos las métricas con f1
    f1_score_aux = f1_score(y_test, y_predict, labels = [0, 1], average = None)
    f1_score_list_class_0.append(f1_score_aux[0])
    f1_score_list_class_1.append(f1_score_aux[1])
    f1_score_weighted_list.append(f1_score(y_test, y_predict, average = 'weighted'))
    f1_score_macro_list.append(f1_score(y_test, y_predict, average = 'macro'))

In [25]:
pd.DataFrame(
    {
        'f1_class0': f1_score_list_class_0,
        'f1_class1': f1_score_list_class_1,
        'f1_weighted': f1_score_weighted_list,
        'f1_macro': f1_score_macro_list  
    },
    index = modelos_guardados,
)

,f1_class0,f1_class1,f1_weighted,f1_macro
single_GradientBoostingClassifier.sav,0.048406,0.957792,0.884104,0.503099
seeing_someone_GradientBoostingClassifier.sav,0.980028,0.000000,0.941698,0.490014
available_GradientBoostingClassifier.sav,0.980002,0.000000,0.941673,0.490001
single_RandomForestClassifier.sav,0.030321,0.956433,0.881389,0.493377
seeing_someone_RandomForestClassifier.sav,0.979689,0.000000,0.941372,0.489844
available_RandomForestClassifier.sav,0.979715,0.000000,0.941397,0.489858
single_AdaBoostClassifier.sav,0.028504,0.957175,0.881924,0.492839
seeing_someone_AdaBoostClassifier.sav,0.980054,0.000000,0.941723,0.490027
available_AdaBoostClassifier.sav,0.980054,0.000000,0.941723,0.490027
single_LogisticRegression.sav,0.037803,0.957348,0.882837,0.497575


In [26]:
# Revisamos la cantidad de valores 1 y 0 por cada vector objetivo
print(f'Cantidad single: {y_test_single.sum()}')
print(f'Cantidad seeing_someone: {y_test_seeing_someone.sum()}')
print(f'Cantidad available: {y_test_available.sum()}')
print(f'Cantidad total: {y_test_single.count()}')

Cantidad single: 18327
Cantidad seeing_someone: 780
Cantidad available: 780
Cantidad total: 19943


Si bien no es el objetivo del notebook, se extraen métricas de rendimiento para los modelos. Se observa que el F1 es mejor siempre en la clase mayoritaria, ya que estamos en un caso en que los 3 vectores objetivos están muy desbalanceados.  

Single tiene muchas observaciones, de modo que el F1 de la clase single = 1 es muy bueno. Sin embargo, single = 0 no tiene un buen rendimiento. Los otros dos vectores objetivos tienen el comportamiento contrario, son muy buenos para la clase 0, que es la mayoritaria.  

Este mal comportamiento se evidencia en el macro avg, el cual castiga más las observaciones mal clasificadas de la categoría minoritaria.